In [3]:
using Pkg
Pkg.activate("..")

  Activating project at `~/Library/CloudStorage/OneDrive-Personal/Documents/Studia/Semestr 8/Algorytmy w inżynierii danych/Projekt/KamienMilowy2/KM2_Piotr_Szczerba`


In [4]:
include("../autodiff/graph.jl")
include("../autodiff/forward.jl")
include("../autodiff/backward.jl")
include("../autodiff/operators.jl")
include("../neuralnet/dataloader.jl")
include("../autodiff/flux_like_api.jl")

using JLD2
using Random
using Statistics
using Printf

X_train = load("../data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../data/imdb_dataset_prepared.jld2", "y_test")

y_train = Float32.(y_train)
y_test = Float32.(y_test)

batch_size = 64

dataset = DataLoader(X_train, y_train, batch_size, shuffle=true)

input_neurons = size(X_train, 1)
hidden_neurons = 32
output_neurons = 1

epochs = 5

ϵ = Constant(1e-7)
binary_cross_entropy_loss(y, ŷ) = mean(Constant(-1.0) .* (y .* log.(ŷ .+ ϵ) .+ (Constant(1.0) .- y) .* log.(Constant(1.0) .- ŷ .+ ϵ)))

binary_cross_entropy_loss (generic function with 1 method)

In [5]:
Dense(input_neurons, hidden_neurons, relu)

Dense{typeof(relu)}(Variable(Float32[0.0010591024 -0.013475468 … -0.006705568 -0.012882979; 0.012632421 -0.0040448876 … -0.0009320052 0.0010995327; … ; 0.0047158375 -0.016759556 … -0.005143863 0.009779765; 0.004992991 0.008362246 … -0.0014324104 0.005771153], nothing, "weight"), Variable([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], nothing, "bias"), Main.relu)

In [6]:
model = Chain(
    Dense(input_neurons, hidden_neurons, relu),
    Dense(hidden_neurons, output_neurons, σ)
)

y = Variable(zeros(1, batch_size), name="y")
x = Variable(zeros(input_neurons, batch_size), name="x")

function loss(model, x, y)
    ŷ = model(x)
    E = binary_cross_entropy_loss(y, ŷ)
    E.name = "loss"
    return E, ŷ
end

otp = setup(Adam(), model)

AdamState(Adam(0.001, 0.9, 0.999, 1.0e-8), Dict{Variable, Tuple{Array, Array, Int64}}(Variable(Float32[0.15169844 0.0692048 … 0.1680429 -0.18622297], nothing, "weight") => ([0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0], 0), Variable([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], nothing, "bias") => ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0), Variable([0.0], nothing, "bias") => ([0.0], [0.0], 0), Variable(Float32[0.005207593 -0.010862097 … 0.014115392 -0.016502006; 0.0009457772 0.006190597 … 0.007992115 -0.017684417; … ; 0.002267362 -0.013136274 … 0.00014214538 -0.004485451; -0.0028275086 -0.0038113743 … -0.009805873 -0.010039179], nothing, "weight") => ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.

In [ ]:
L, ŷ_node = loss(model, x, y)
graph = topological_sort(L)

for epoch in 1:epochs
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    t = @elapsed begin
        for (xb, yb) in dataset
            current_batch_size = size(xb, 2)

            x.output .= xb
            y.output .= yb

            lval = forward!(graph)

            for param in trainable(model)
                param.gradient = nothing
            end
            backward!(graph)

            update!(otp, model)

            ŷ = ŷ_node.output
            predictions = ŷ .> 0.5
            targets = y.output .> 0.5
            total_correct += count(predictions .== targets)
            total_loss += lval[1] * current_batch_size
            total_samples += current_batch_size
        end
    end

    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (loss: %.4f, acc: %.4f)",
        epoch, t, avg_loss, avg_acc))

end

Epoch: 1 (11.61s) 	Train: (loss: 0.6452, acc: 0.7784)
Epoch: 2 (2.66s) 	Train: (loss: 0.4577, acc: 0.9169)
Epoch: 3 (2.49s) 	Train: (loss: 0.2970, acc: 0.9435)
Epoch: 4 (2.44s) 	Train: (loss: 0.2029, acc: 0.9639)
Epoch: 5 (2.83s) 	Train: (loss: 0.1447, acc: 0.9765)
